In [1]:
import numpy as np
import os
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Flatten, Reshape, Input, InputLayer
from keras.models import Sequential, Model
from keras.callbacks import TensorBoard
import cv2
import matplotlib.pyplot as plt

2022-07-11 13:08:01.957359: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:
2022-07-11 13:08:01.957394: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
folder_path = 'FOLDER_PATH'

In [3]:
def load_data(path):
    output_data = []
    for file in os.listdir(path):
        if not file.endswith(('.png', '.jpg', '.jpeg', '.JPG')) or 'label' in file:
            continue
        image = cv2.imread(os.path.join(path,file))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (256, 256)) 
        output_data.append(image)
    return np.array(output_data)

X = load_data(folder_path)

In [5]:
def build_autoencoder(img_shape, code_size):
    # The encoder
    encoder=Sequential()
    encoder.add(InputLayer(img_shape))
    encoder.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    encoder.add(MaxPooling2D((2,2), padding='same'))
    encoder.add(Conv2D(32, (3,3),activation='relu',padding='same'))
    encoder.add(MaxPooling2D((2,2), padding='same'))
    encoder.add(Conv2D(16, (3,3),activation='relu',padding='same'))
    encoder.add(MaxPooling2D((2,2), padding='same'))
    encoder.add(Flatten())
    encoder.add(Dense(code_size))

    # The decoder
    decoder=Sequential()
    decoder.add(InputLayer((code_size,)))
    decoder.add(Dense(np.prod(16384)))
    decoder.add(Reshape((32,32,16)))
    decoder.add(Conv2D(16, (3,3), activation='relu', padding='same'))
    decoder.add(UpSampling2D((2,2)))
    decoder.add(Conv2D(32, (3,3), activation='relu', padding='same'))
    decoder.add(UpSampling2D((2,2)))
    decoder.add(Conv2D(64, (3,3), activation='relu', padding='same'))
    decoder.add(UpSampling2D((2,2)))
    decoder.add(Conv2D(3, (3,3), activation='relu', padding='same'))
    decoder.add(Reshape((256, 256, 3)))

    return encoder, decoder


IMG_SHAPE = X.shape[1:]
encoder, decoder = build_autoencoder(IMG_SHAPE, 32)

inp = Input(IMG_SHAPE)
bottleneck = encoder(inp)
reconstruction = decoder(bottleneck)

autoencoder = Model(inp,reconstruction)
autoencoder.compile(optimizer='adamax', loss='mse',metrics=['accuracy'])

autoencoder.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 32)                549200    
                                                                 
 sequential_1 (Sequential)   (None, 256, 256, 3)       567859    
                                                                 
Total params: 1,117,059
Trainable params: 1,117,059
Non-trainable params: 0
_________________________________________________________________


2022-07-11 13:08:18.891379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cv/naveen_venv/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:
2022-07-11 13:08:18.891468: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cv/naveen_venv/lib/python3.7/site-packages/cv2/../../lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:/usr/local/cuda-10.2/lib64:
2022-07-11 13:08:18.891542: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such fi

In [6]:
tensorboard_callback = TensorBoard(log_dir="LOG_DIR")

In [15]:
autoencoder.fit(X, X, epochs=1000,batch_size=18,callbacks=[tensorboard_callback], shuffle=True)
autoencoder.save('image_encoder.h5')

Epoch 1/1000
2/2 [==============================] - 4s 2s/step - loss: 894.9847 - accuracy: 0.5042
Epoch 2/1000
2/2 [==============================] - 4s 2s/step - loss: 888.5635 - accuracy: 0.5035
Epoch 3/1000
2/2 [==============================] - 4s 2s/step - loss: 882.3088 - accuracy: 0.5036
Epoch 4/1000
2/2 [==============================] - 4s 2s/step - loss: 875.3348 - accuracy: 0.5039
Epoch 5/1000
2/2 [==============================] - 4s 2s/step - loss: 870.0178 - accuracy: 0.5033
Epoch 6/1000
2/2 [==============================] - 4s 2s/step - loss: 861.9439 - accuracy: 0.5031
Epoch 7/1000
2/2 [==============================] - 4s 2s/step - loss: 859.6338 - accuracy: 0.5041
Epoch 8/1000
2/2 [==============================] - 4s 2s/step - loss: 851.0341 - accuracy: 0.5036
Epoch 9/1000
2/2 [==============================] - 4s 2s/step - loss: 846.6522 - accuracy: 0.5038
Epoch 10/1000
2/2 [==============================] - 4s 2s/step - loss: 840.4077 - accuracy: 0.5044
Epoch 11/

In [17]:
pred=autoencoder.predict(X)
image = cv2.cvtColor(pred[0], cv2.COLOR_BGR2RGB)
cv2.imwrite('test.png',image)

2/2 [==============================] - 1s 61ms/step


True